In [1]:
#Importing all dependencies

# Data Manipulation - Pandas, Numpy, Math
import pandas as pd
import numpy as np 
import os
import math
import itertools
import json
import requests
#from Functions_TOS import * 


# Web Scraping Packages - Selenium, Beautiful, Request

import requests
from bs4 import BeautifulSoup
from selenium import webdriver #Externally Downloaded
from selenium.webdriver.common.keys import Keys #Externally Downloaded
from selenium.webdriver.chrome.options import Options #Externally Downloaded

#HTML Parser 
#from html_table_parser.parser import HTMLTableParser #INFO: We don't really need this for now as it's only for cosmetic view of table in Jupyter Notebooks.
from lxml import html
#import pandas_datareader.data as web

# Stocks Data Pulling 
#from pystocktwits import * #Error: Need Github Repo. Externally Downloaded
#from waybackpy import Cdx # Error: Can't include Cdx
#from waybackpy import * #Externally Downloaded

# Date, Time and Calendar Packages

from datetime import date, timedelta, datetime, time
import time
import datetime
#import pandas_market_calendars as mcal #Externally Downloaded
from calendar import monthrange

import warnings
warnings.filterwarnings('ignore')


C:\Users\lukshkumar\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [25]:
def getcustom_global_re_indexing(data,number):
	"""Adds new index as a multiple of a given number 
	Args: 
	data : what data to add index to 
	number : which number's multiples to use"""
	numbers = len(data)
	numbers_ = []
	for  i in range(1,numbers+1):
		numbers_.append(i*number)
	data['Global_Index'] = numbers_
	return data 

def get_date_from_global_re_index(data,global_re_indexing_value):
	"""Gets Date from global index. If exact match not found round to next upper number  . 
	e.g if we are using the multiples of 3 
	and we enter 5 it will return the date at index of 6
	Args: 
	data : what data to use
	global_re_indexing_value : what index to look for ."""
	df_ = data[data['Global_Index'] == global_re_indexing_value]["Date"]
	if len(df_) == 0 : 
		working_numbers = list(data['Global_Index'])
		greter_than = []
		for _ in working_numbers : 
			if _ > global_re_indexing_value: 
				greter_than.append(_)
		greter_than =sorted(greter_than)
		df_ = data[data["Global_Index"] == greter_than[0]]["Date"].tolist()[0]
	return df_ 

def SimpleMovingAvg(data, price, length):
	"""Gets simple moving average for the given data
	Argumnets : 
	data : data to work with 
	price : column 
	length : how many days to work for """
	symbols = data["Symbol"].unique()
	sma = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].rolling(length).mean():
			sma.append(__)
	return sma

def get_highest_column_value(data,columns,offset):
	"""Returns a column with highest values for that given row
	data: data to work with 
	columns :What columns to work with
	offset : Wether to get second highest,third highest...."""
	df = data[columns]
	dataFRAME  = df.values.tolist()
	new_column = []
	for row in dataFRAME : 
		new_column.append(sorted(row,reverse = True)[offset])
	return new_column

def get_lowest_column_value(data,columns,offset):
	"""Returns a column with lowest values for that given row
	data: data to work with 
	columns :What columns to work with
	offset : Wether to get second lowest,third lowest...."""
	df = data[columns]
	dataFRAME  = df.values.tolist()
	new_column = []
	for row in dataFRAME : 
		new_column.append(sorted(row)[offset])
	return new_column

def get_sequence(data,columns,strings):
	"""It returns a column based on  sorted values in descending order by matching with given letters
	data:data to work with 
	columns : what columns to work with 
	 strings : what strings to assign to the column """
	df = data[columns]
	sequences  = []
	for row in df.iterrows():
			list_ = row[1].to_list()
			res = {strings[i]: list_[i] for i in range(len(strings))}
			res = {k: v for k, v in sorted(res.items(), key=lambda item: item[1],reverse = True)}
			sequences.append("".join(res.keys()))
	return sequences

def get_permutation_value(data,column):
	"""Returns a new column with permuation order of data
	data : data to work with 
	columns : columns to work with """
	data = data[column]
	s = data.iloc[0]
	nums = list(s)
	permutations = {"".join(element): index for index,element in enumerate(list(itertools.permutations(nums)))}  
	permutations_ = []
	for value in  data: 
		permutations_.append(permutations[value])
	return permutations_

def get_custom_symbol_parsed_re_indexing(data,number):
	"""Adds custom index for each symbol by given number's multiples 
	Args:
	data : what data to add index to 
	number: which number's multiple to add as new index """
	symbols = data["Symbol"].unique()
	numbers = []
	for _  in symbols: 
		df_ = data[data['Symbol'] == _]
		for  i in range(1,len(df_)+1):
			numbers.append(i*number)
	data['Symbol_re_indexing'] = numbers
	return data

def get_date_from_custom_symbolparsed_re_indexing(data,symbol,number):
	"""Gets date from custom symbol index : 
	Args : 
	data :what dat to look in 
	symbol : what symbol to look for 
	number :what index to get date from """
	symbol = symbol.upper()
	data = data[data['Symbol'] == symbol]
	df_ = data[data['Symbol_re_indexing']== number]["Date"]
	if len(df_) == 0 : 
		working_numbers = list(data['Symbol_re_indexing'])
		greter_than = []
		for _ in working_numbers : 
			if _ > number: 
				greter_than.append(_)
		greter_than =sorted(greter_than)
		df_ = data[data["Symbol_re_indexing"] == greter_than[0]]["Date"].tolist()[0]
	return df_

def get_date_from_re_indexing(data,starting_number, ending_number):
    """
    starting_number = starting_number + 5 - (starting_number %  5)
    ending_number = ending_number + 5 - (ending_number %  5)
    print(starting_number)
    print(ending_number)
    df_ = data[data['Symbol_re_indexing']== starting_number]
    df_end = data[data['Symbol_re_indexing']== ending_number]
    """
    df_ = data.iloc[[starting_number], :]
    df_ = df_.append(data.iloc[[ending_number], :])
    if 'Time' in df_.columns:
        df_ = df_[["Date", "Time"]]
    else:
        df_ = df_["Date"]
    
    if len(df_) == 0 : 
        working_numbers = list(data['Symbol_re_indexing'])
        greter_than = []
        for _ in working_numbers : 
            if _ > number: 
                greter_than.append(_)
        greter_than =sorted(greter_than)
        df_ = data[data["Symbol_re_indexing"] == greter_than[0]]["Date"].tolist()[0]
    return df_

def get_data_from_re_indexing(data,starting_number, ending_number):
	df_ = data[(data['Symbol_re_indexing'] >= starting_number) & (data['Symbol_re_indexing'] <= ending_number)]
	if len(df_) == 0 : 
		working_numbers = list(data['Symbol_re_indexing'])
		greter_than = []
		for _ in working_numbers : 
			if _ > number: 
				greter_than.append(_)
		greter_than =sorted(greter_than)
		df_ = data[data["Symbol_re_indexing"] == greter_than[0]]["Date"].tolist()[0]
	return df_

def get_start_end_only_data(data,start_date, end_date=None):
    """Gets all data between two given dates 
    Args: 
    data :What data to perform the calculations on 
    start_date : The start date for the data 
    end_date : the end date for the data"""
    #Date =list(data.Date)
    #date = []
    #for _ in Date: 
    #    print(_)
    #	_ = str(_)
    #	date.append(f'{_[:4]}-{_[4:6]}-{_[6:]}')
    #print(date)
    
    data.Date  = pd.to_datetime(data.Date)
    if end_date :
        mask = (data['Date'] >= start_date) & (data['Date'] <= end_date)
    else: 
        mask = (data['Date'] >= start_date)
    return data.loc[mask]

def getquery_blast_txt_file (data,symbol,column,start_date,end_date):
	""" Makes a txt file  by concating all values for  a given columm between given indexes
	data: data to work with 
	symbol : symbol to get data for 
	column : column to get data for 
	start_date : start date
	end_date : end date """
	data  = get_start_end_only_data(data,start_date,end_date)
	total_rows_in_query_dataframe  = data.shape[0]    
	start = 0 
	end = len(data)-1
	file = open(f"Blast/query_{symbol}.txt","w")
	file.write(f">query_{symbol}\n")
	data= list(data[data['Symbol'] == symbol][column])
	data  = data[start:end]    
	data_ = []
	for _ in data : 
		data_.append(str(_))
	data = "".join(data_)
	file.write(data)
	file.close()
	return total_rows_in_query_dataframe

    
def getsubject_blast_txt_file (data,symbol,column,start_date,end_date, query_start_date, query_end_date):
    """ Makes a txt file  by concating all values for  a given columm between given indexes
    data: data to work with 
    symbol : symbol to get data for 
    column : column to get data for 
    start_date : start date
    end_date : end date """
    data  = get_start_end_only_data(data,start_date,end_date)    
    
    # Making sure that we don't get the query data within subject itself because we don't want the same match of query data
    # with itself. Therefore removing the query data from this subject.
    
    data_after_query = data.copy(deep=True)
    mask_before_query = (data['Date'] < query_start_date)
    mask_after_query = (data['Date'] > query_end_date)
    data = data.loc[mask_before_query]
    data_after_query = data_after_query.loc[mask_after_query]
    data = data.append(data_after_query)
    data = data.reset_index(drop = True)
    
    subject_starting_date = str(data.loc[0, "Date"])
    subject_ending_date = str(data.loc[len(data.index) - 1, "Date"])
    
    print("Trimmed Subject Starting Date: ", subject_starting_date)
    print("Trimmed Subject Ending Date: ", subject_ending_date)
    
    start = 0 
    end = len(data)-1
    file = open(f"Blast/subject_{symbol}.txt","w")
    file.write(f">subject_{symbol}\n")
    data= list(data[data['Symbol'] == symbol][column])
    data  = data[start:end]
    data_ = []
    for _ in data : 
        data_.append(str(_))
    data = "".join(data_)
    file.write(data)
    file.close()
    
def SimpleMovingAvg(data, price, length):
	"""Gets simple moving average for the given data
	Argumnets : 
	data : data to work with 
	price : column 
	length : how many days to work for """
	symbols = data["Symbol"].unique()
	sma = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].rolling(length).mean():
			sma.append(__)
	return sma

def Variance(data, price, length):
	"""Gets Variacne for the given data
	Argumnets : 
	data : data to work with 
	price : column 
	length : how many days to work for """
	symbols = data["Symbol"].unique()
	sma = []
	for _ in symbols: 
		for __ in data[data['Symbol'] == _][price].rolling(length).var():
			if __ == 0 : 
				sma.append(1)
			else :
				sma.append(__)
	return sma

def Start_Range(data,column):
	"""Gets some math stuff
	Arguments: 
	data :What data to get math stuff for 
	column : what column to get the math stuff for """
	amount = 2
	values = [None]
	for step in range( 1,len(data)): 
		values.append(data['2_day_average'].iloc[step] - math.sqrt( abs(data["Variance"].iloc[step]  - (data["2_day_average"].iloc[step] - data[column].iloc[step-1] )**2  )) )		
	return values

def Start_Range_Recursion(data,column):
	"""Gets some math stuff
	Arguments: 
	data :What data to get math stuff for 
	column : what column to get the math stuff for """
	for step in range( 1,len(data)): 
		values=data['2_day_moving_average_recursion'].iloc[step] - math.sqrt(abs(   data["Variance_from"].iloc[step]  - (data["2_day_moving_average_recursion"].iloc[step] - data[column].iloc[step-1] )**2 ) )
	return values

def End_Range(data,column):
	"""Gets some math stuff
	Arguments: 
	data :What data to get math stuff for 
	column : what column to get the math stuff for """
	amount = 2
	values = [None]
	for step in range( 1,len(data)): 
		values.append(math.sqrt(abs( data['Variance'].iloc[step]  -( data["2_day_average"].iloc[step]  -data[column].iloc[step-1])**2 )) +data["2_day_average"].iloc[step])		
	return values

def End_Range_Recursion(data,column):
	"""Gets some math stuff
	Arguments: 
	data :What data to get math stuff for 
	column : what column to get the math stuff for """
	amount = 2
	for step in range( 1,len(data)): 
		values= math.sqrt(abs (data['Variance_from'].iloc[step]  -( data["2_day_moving_average_recursion"].iloc[step]  -data[column].iloc[step-1])**2 )) +data["2_day_moving_average_recursion"].iloc[step]
	return values

def Factor_Fraction(data,column):
	"""Gets some math stuff
	Arguments: 
	data :What data to get math stuff for 
	column : what column to get the math stuff for """
	values = [None]
	for step in range( 1,len(data)): 
		if abs(data['Close'].iloc[step] - data['Start_Range'].iloc[step]) < abs(data['Close'].iloc[step] - data['End_Range'].iloc[step]):
			values.append(0)
		else: 
			values.append(1)
	return values

def fetchMarketOpenDays():
    r = requests.get("https://financialmodelingprep.com/api/v3/is-the-market-open?apikey=aa8b4631b2ad1b2704741d1a2d6a2611")

    last_year_records = r.json()["stockMarketHolidays"][-1]
    last_year_records.pop("year")

    holidays_list = []
    for i in last_year_records:
           holidays_list.append(datetime.datetime.strptime(last_year_records[i], '%Y-%m-%d'))

    return holidays_list

def ticks(dt):
    return (dt - datetime.datetime(1, 1, 1)).total_seconds()
    
def get_how_to_append_end_data (data_file_from, column_from, index_start_from, index_end_from, data_file_to, index_start_to, append_type ):
    """" Does fancy math stuff to help predict something 
    Arguments : 
    data_file_from : data file from where we take teh da ta
    column_from : column from whree we take the data
    index_strart_from : index of the start for the dat_file_from file 
    index_end_from : index of the end for the dat_file_from file 
    data_file_to : data to append the data to 
    index_start_to : index of the data file to append the data to 
    append_type : wether to use new predicted value for the append or the other 
    """
    #print("Index Start From: ", index_start_from)
    #print("Index End From: ", index_end_from)
    #print("Data File From Shape: ", data_file_from.shape)
    
    #print("-----------------------------Data File From----------------------------------")
    
    data_file_from =data_file_from.iloc[index_start_from-1: index_end_from]
    #print(data_file_from)
    start_date  = data_file_from["Date"].iloc[0]
    end_date  = data_file_from["Date"].iloc[-1]
    data_file_from['2_day_average'] = SimpleMovingAvg(data_file_from,column_from,2)
    data_file_from['Variance'] = Variance(data_file_from,column_from,2)
    data_file_from["Start_Range"] = Start_Range(data_file_from ,column_from)
    data_file_from["End_Range"] = End_Range(data_file_from ,column_from)
    data_file_from["Factor_Fraction"] = Factor_Fraction(data_file_from ,column_from)
    data_file_from.drop(index=data_file_from.index[0], 
            axis=0, 
            inplace=True)
    data_file_to = data_file_to.iloc[index_start_to-1:]
    #print("-----------------------------Data File To----------------------------------")
    #print(data_file_to)
    len_data_files_to = len(data_file_to)
    close_recursion = [None]*len(data_file_to)    
    close_recursion[-2] = data_file_to[column_from].iloc[-2]
    data_file_to[f"{column_from}_Recurrsion"]  = close_recursion
    varaiance = [] 
    for x in data_file_from["Variance"]:
        varaiance.append(x)
    factor_Fraction = [] 
    for x in data_file_from["Factor_Fraction"]:
        factor_Fraction.append(x)
    data___ = pd.DataFrame({"Factor_Fraction_from":factor_Fraction,"Variance_from":varaiance})
    data_file_to['2_day_average'] =SimpleMovingAvg(data_file_to,column_from,2)
    moving_average_recursion = [None]*len(data_file_to)
    moving_average_recursion[-1] = data_file_to["2_day_average"].iloc[-1]
    data_file_to["2_day_moving_average_recursion"]  = moving_average_recursion
    #print(data_file_to)    
    data_file_to = pd.concat([data_file_to,pd.DataFrame(data___)],ignore_index=False)
    #print(data_file_to)
    data_file_to['Variance_from'] = data_file_to['Variance_from'].shift(-1)
    data_file_to['Factor_Fraction_from'] = data_file_to['Factor_Fraction_from'].shift(-1)
    data_file_to = data_file_to[[column_from,"Date",'Factor_Fraction_from',"Variance_from","2_day_moving_average_recursion","Symbol",f"{column_from}_Recurrsion"]]
    data_file_to["Start_Range"] = [None]*len(data_file_to)
    data_file_to["End_Range"] = [None]*len(data_file_to)
    data_file_to = data_file_to.dropna(how = "all")
    #print("-----------------------------Data File To----------------------------------")
    #print(data_file_to)
    #data_file_from.to_csv('Test-Append.csv')
    for working_index in range(len_data_files_to-1,len(data_file_to)):
            #print(data_file_to)
            values  = Start_Range_Recursion(data_file_to.iloc[working_index-1:working_index+1],f"{column_from}_Recurrsion")
            idx = data_file_to.iloc[working_index].name
            data_file_to.at[idx,"Start_Range"] = values
            values  = End_Range_Recursion(data_file_to.iloc[working_index-1:working_index+1],f"{column_from}_Recurrsion")
            idx = data_file_to.iloc[working_index].name
            data_file_to.at[idx,"End_Range"] = values
            if  data_file_to.iloc[working_index]["Factor_Fraction_from"] == 0: 
                data_file_to.at[data_file_to.iloc[working_index].name,f"{column_from}_Recurrsion"] = data_file_to.iloc[working_index]["Start_Range"]
            else : 
                data_file_to.at[data_file_to.iloc[working_index].name,f"{column_from}_Recurrsion"] = data_file_to.iloc[working_index]["End_Range"]
            moving_average = data_file_to.iloc[working_index-1:working_index+1][f"{column_from}_Recurrsion"].mean()
            try: 
                data_file_to.at[data_file_to.iloc[working_index+1].name,"2_day_moving_average_recursion"] =moving_average
            except:
                pass
    data_file_to = data_file_to[[column_from,f'{column_from}_Recurrsion',"Symbol","Date"]]
    Dates_to_work_with = list(data_file_to.dropna(subset = ["Date"]).Date)
    number_of_misisng_dates = data_file_to.Date.isna().sum()
    Symbols_to_work_with = list(data_file_to.dropna(subset = ["Symbol"]).Symbol)
    close_recurstion_data = list(data_file_to.dropna(subset=[f"{column_from}_Recurrsion"])[f"{column_from}_Recurrsion"])
    close_data = list(data_file_to.dropna(subset = [column_from])[column_from])
    close_recursion_data = close_recurstion_data[2:]
    close_data.extend(close_recursion_data)
    #print("-----------------------------Data File To----------------------------------")
    #print(data_file_to)
    
    for number in range(number_of_misisng_dates):
        try:
            date_time_obj = Dates_to_work_with[-1] + datetime.timedelta(1)
        except:
            date_time_obj = Dates_to_work_with[-1] + datetime.timedelta(1)          
        Dates_to_work_with.append(date_time_obj)
        Symbols_to_work_with.append(Symbols_to_work_with[-1])

    data_file_to["Date"] = Dates_to_work_with
    data_file_to["Symbol"] = Symbols_to_work_with
    data_file_to[column_from] = close_data
    del data_file_to[f"{column_from}_Recurrsion"]
    #print(data_file_to.to_csv("Test.csv"))
    
    # Checking the Authenticated dates:
    start_of_trading_day = data_file_to.loc[0,"Date"]
    end_of_trading_day = data_file_to.iloc[-1]["Date"]
    start_of_trading_day = datetime.datetime.strftime(start_of_trading_day, '%Y-%m-%d')
    start_of_trading_day = datetime.datetime.strptime(start_of_trading_day, '%Y-%m-%d')
    
    #print(start_of_trading_day)
    #print(type(start_of_trading_day))
    day_delta = datetime.timedelta(days=1)
    dates_of_market_open = []
    i = 0
    holidays_list = fetchMarketOpenDays()
    while(len(dates_of_market_open) != data_file_to.shape[0]):
        current_date = start_of_trading_day + i*day_delta
        if(((current_date not in holidays_list)) and (current_date.weekday() < 5)):
            dates_of_market_open.append(current_date)
        i += 1
    data_file_to["Date"] = Dates_to_work_with
    #print(dates_of_market_open)
        
    #print(some_error)
    return data_file_to


# Blast - Daily Data 

### Set Parameters

In [29]:
query_starting_date = "20220417"
query_ending_date = "20220517"

take_entire_data_for_subject = True

#Only specify starting and ending dates if you are not taking the entire data in subject.
subject_starting_date = "19900102"
subject_ending_date = "20220503"

# We have 250 rows in query and we want 1/10th of the query in predicted so we put 10 in ratio_of_predicted_entries. 
# The Predicted data will be 1/10th of the size of the query data. This 10 here is the value of denominator.
ratio_of_predicted_entries = 1

# Number of top results to match from blast.
top_n_blast_results = 10

# Run for all tickers within a directory
run_for_all_tickers_within_a_directory = False

# If run_for_all_tickers_within_a_directory = false then mention the list of tickers you want to run the program for.
data_files = ["SPX"]

#If running on Live system in real-time then set it to false. This will make unique files and won't override one on another.
dev_testing = True

#List of Moving Averages

list_of_MA_list = [
    [5, 20, 50, 100, 200],
    [9, 19, 41, 88, 129],
    [2, 3, 4, 6, 13]
    
] 

# list_of_MA_list = [
#     [5, 20, 50, 100, 200]
#     ] 

# [10, 18, 52, 71, 75]


## Daily Data Blast Prediction

In [30]:
# Number of letters in re-indexed columns
number_of_letters = 0
for each_MA_list in list_of_MA_list:
    number_of_letters += len(each_MA_list) 

blast_results_metric = "Identity"
datetime_ticks = ""
if(dev_testing):
    datetime_ticks = "-" + str(ticks(datetime.datetime.utcnow()))
list_of_MA_list_string = str(list_of_MA_list)
blast_index_start = 100
blast_index_end = 200

folder_path = "Tickers/"
blast_folder_path = "Blast/"

if(run_for_all_tickers_within_a_directory):
    data_files = os.listdir(folder_path)
else:
    for each_file_index_in_directory in range(len(data_files)):
        data_files[each_file_index_in_directory] = data_files[each_file_index_in_directory] + ".csv"

subject_files = os.listdir(blast_folder_path)
#Reading the Blast-Tickers File

blast_tickers = pd.read_csv("Blast-Tickers.csv")
blast_tickers = blast_tickers["Tickers"].values.tolist()
print("Blast Ticker Received From File: ", blast_tickers)

for data_file in data_files:

    print("---------------- Filename: " + data_file + " ----------------")
    symbol_name = data_file.replace(".csv","")
    df_daily_data = pd.read_csv(folder_path + data_file)
    df_subject_data_to_merge_with_prediction_data = df_daily_data.copy(deep=True)
    
    if(take_entire_data_for_subject):
        subject_starting_date = str(df_daily_data.loc[0, "Date"])
        subject_ending_date = str(df_daily_data.loc[len(df_daily_data.index) - 1, "Date"])
        print("Subject Start Date: ", subject_starting_date)
        print("Subject Ending Date: ", subject_ending_date)

    #Making a separate folder for each ticker.
    if(not os.path.isdir(blast_folder_path + "Tickers/" + symbol_name)):
        os.mkdir(blast_folder_path + "Tickers/" + symbol_name)
    blast_resultant_filename = blast_folder_path + "Tickers/" + symbol_name + "/results_query_" + symbol_name
    current_subject_filename = "subject_" + symbol_name + ".txt"
    current_query_filename = "query_" + symbol_name + ".txt"

    df_daily_data = df_daily_data.astype({"Date": str})

    getcustom_global_re_indexing(df_daily_data,number_of_letters)

    MA_list_count = 1
    for each_MA_list in list_of_MA_list:

        list_of_each_MA_value_column_name = []
        each_MA_list_sequence_name = "sequence" + str(MA_list_count)
        for each_MA_value in each_MA_list:

            each_MA_value_column_name = str(each_MA_value) + "_day_sma"
            list_of_each_MA_value_column_name.append(each_MA_value_column_name)
            #SMA Daily
            df_daily_data[each_MA_value_column_name] = SimpleMovingAvg(df_daily_data, "Close", each_MA_value)

        df_daily_data[each_MA_list_sequence_name] = get_sequence(df_daily_data,list_of_each_MA_value_column_name,["A", "B", "C", "D", "E"])
        MA_list_count += 1

    get_custom_symbol_parsed_re_indexing(df_daily_data,number_of_letters)

    # Merging all list of MA columns into one single column named as sequence
    consolidated_sequence = []
    for i in range(len(df_daily_data)):
        each_row_sequence = ""
        for j in range(1, len(list_of_MA_list)+1):
            j_column_name = "sequence" + str(j)
            each_row_sequence += df_daily_data.loc[i, j_column_name]
        consolidated_sequence.append(each_row_sequence)

    df_daily_data["sequence"] = consolidated_sequence

    #df_daily_data.to_csv("Testing-" + symbol_name +".csv")

    #getquery_blast_txt_file (data,symbol,column,start_date,end_date)
    total_rows_in_query_dataframe = getquery_blast_txt_file (df_daily_data,symbol_name,"sequence",query_starting_date, query_ending_date)
    print("Total Rows in Query: ", total_rows_in_query_dataframe)
    number_of_predicted_entries = int(total_rows_in_query_dataframe / ratio_of_predicted_entries)
    print("Predicted Rows: ", number_of_predicted_entries)

    #getsubject_blast_txt_file (data,symbol,column,start_date,end_date)
    getsubject_blast_txt_file (df_daily_data,symbol_name,"sequence",subject_starting_date, subject_ending_date, query_starting_date, query_ending_date)

    #BLAST INTEGRATION - FOR CURRENT SYMBOL
    !makeblastdb -in C:\Users\lukshkumar\Desktop\Blast\$current_subject_filename -dbtype prot -out C:\Users\lukshkumar\Desktop\Blast\DB
    !blastp -query C:\Users\lukshkumar\Desktop\Blast\$current_query_filename -db C:\Users\lukshkumar\Desktop\Blast\DB -out C:\Users\lukshkumar\Desktop\Blast\output.txt -outfmt 6 -max_hsps $top_n_blast_results

    blast_result_csv_file = pd.DataFrame(columns = ["Ticker", "Query", "Subject", "Identity", "Alignment Length", "MisMatches", "Gap Opens", "Query Start", "Query End", "Subject Start", "Subject End", "Evalue", "Bit Score"])
    file = open(blast_folder_path  + "output.txt","r")
    lines = file.readlines()
    file.close()
    for line in lines:
        blast_file_result = line.replace("\n","").split("\t")
        blast_file_result.insert(0,symbol_name)
        blast_result_csv_file.loc[len(blast_result_csv_file.index)] = blast_file_result

    #BLAST INTEGRATION - FOR OTHER SYMBOLS IN THE LIST
    for blast_ticker in blast_tickers:
        if(blast_ticker != symbol_name):
            blast_subject_filename = "subject_" + blast_ticker + ".txt"

            if(os.path.isfile(blast_folder_path +  blast_subject_filename)):
                # MAKE BLAST DB FROM THE SUBJECT
                !makeblastdb -in C:\Users\lukshkumar\Desktop\Blast\$blast_subject_filename -dbtype prot -out C:\Users\lukshkumar\Desktop\Blast\DB
                # RUN BLAST TO FIND THE MATCH
                !blastp -query C:\Users\lukshkumar\Desktop\Blast\$current_query_filename -db C:\Users\lukshkumar\Desktop\Blast\DB -out C:\Users\lukshkumar\Desktop\Blast\output.txt -outfmt 6 -max_hsps $top_n_blast_results
                # APPEND DATA IN CSV FILE
                file = open(blast_folder_path  + "output.txt","r")
                lines = file.readlines()
                file.close()
                for line in lines:
                    blast_file_result = line.replace("\n","").split("\t")
                    blast_file_result.insert(0,blast_ticker)
                    blast_result_csv_file.loc[len(blast_result_csv_file.index)] = blast_file_result


    blast_result_csv_file[blast_results_metric] = pd.to_numeric(blast_result_csv_file[blast_results_metric])
    blast_result_csv_file = blast_result_csv_file.sort_values(by=[blast_results_metric], ascending = False).reset_index(drop=True)
    blast_result_csv_file.to_csv(blast_resultant_filename + ".csv", index = False)

    #Creating another column for the filename for easier reference.
    blast_result_csv_file["filename"] = ""
    #Making sure that the top_n_blast_results are greater than the number of results in blasts file in case blast file has 
    #One result only and top results are two then it should not break.
    if(top_n_blast_results > blast_result_csv_file.shape[0]):
        top_n_blast_results = blast_result_csv_file.shape[0]
    for i in range(top_n_blast_results):
        print("I: ", i)
        blast_ticker_name = blast_result_csv_file.iloc[i,0]
        blast_identity_percentage = round(blast_result_csv_file.iloc[i,3], 0)
        blast_index_start = int(blast_result_csv_file.iloc[i,9])
        blast_index_end = int(blast_result_csv_file.iloc[i,10])

        #print("Blast Index Start: ", blast_index_start)

        source_dataframe  = pd.read_csv(folder_path + blast_ticker_name + ".csv")
        source_dataframe_total_rows = source_dataframe.shape[0]
        # Appending the data from Matched Ticker to the Target Ticker Data File.
        target_dataframe = get_how_to_append_end_data(source_dataframe,"Close",math.ceil(blast_index_end/number_of_letters) + 1,math.ceil(blast_index_end/number_of_letters) + number_of_predicted_entries + 2,df_daily_data,df_daily_data.shape[0] - 1,True)

        # Adding additional fields in target dataframe (predicted data). 
        target_dataframe["Open"] = target_dataframe["Close"]
        target_dataframe["High"] = target_dataframe["Close"]
        target_dataframe["Low"] = target_dataframe["Close"]
        target_dataframe["Volume"] = 0
        target_dataframe["Date"] = target_dataframe["Date"].dt.strftime('%Y%m%d')

        #Dropping the first two unnecessary records from the target dataframe that were only used to create predicted data.
        target_dataframe = target_dataframe.iloc[2:,:]

        consolidated_filename_date_portion = str(target_dataframe.loc[0, "Date"])
        # Appending the source data into target dataframe in order to make up a combinded file having previous and predicted data.
        consolidated_source_plus_target_df = df_subject_data_to_merge_with_prediction_data.append(target_dataframe)
        consolidated_source_plus_target_df.reset_index(drop = True)

        # Date Against the Index Matched from Blast
        df_date_we_match_found = get_date_from_re_indexing(source_dataframe,(math.ceil(blast_index_start/number_of_letters) -1), (math.ceil(blast_index_end/number_of_letters)))
        
        consolidated_source_plus_target_filename = symbol_name + consolidated_filename_date_portion + "D" + str(ratio_of_predicted_entries) + "V" + str(i + 1) + datetime_ticks + ".csv"
        consolidated_source_plus_target_filename_with_path = blast_folder_path + "Tickers/" + symbol_name + "/" + consolidated_source_plus_target_filename

        consolidated_source_plus_target_df.to_csv(consolidated_source_plus_target_filename_with_path, index = False)

        print("Match is Found in: ", blast_ticker_name, "    Ratio: ", blast_identity_percentage)
        print("Date Start: ", df_date_we_match_found.iloc[0])
        print("Date End: ", df_date_we_match_found.iloc[1])
        print("-----------------------------------------------------------------------------")
        
        #Adding new columns into result_query csv file
        blast_result_csv_file.loc[i, "filename"] =  consolidated_source_plus_target_filename
        blast_result_csv_file.loc[i, "match start date"] =  df_date_we_match_found.iloc[0]
        blast_result_csv_file.loc[i, "match end date"] =  df_date_we_match_found.iloc[1]
        blast_result_csv_file.loc[i, "subject size"] =  source_dataframe_total_rows
        blast_result_csv_file.loc[i, "match settings"] =  list_of_MA_list_string
        blast_result_csv_file.loc[i, "query size"] =  total_rows_in_query_dataframe
        blast_result_csv_file.loc[i, "prediction ratio"] =  '"1/' + str(ratio_of_predicted_entries) + '"'
        
#Saving the updated file with Date added in the filename so that we have a separate blast result file for each day.
blast_result_csv_file.to_csv(blast_resultant_filename + "_" + consolidated_filename_date_portion + "_D" + datetime_ticks + ".csv", index = False)
#Removing the generic file created above to make sure we don't have duplicate data.
os.remove(blast_resultant_filename + ".csv")

print("---------------- Program Execution Completed Successfully ----------------")

Blast Ticker Received From File:  ['AAPL', 'TSLA', 'AMZN', 'GOOG', 'NVDA', 'MSFT']
---------------- Filename: SPX.csv ----------------
Subject Start Date:  19280103
Subject Ending Date:  20220517
Total Rows in Query:  22
Predicted Rows:  22
Trimmed Subject Starting Date:  1928-01-03 00:00:00
Trimmed Subject Ending Date:  2022-04-14 00:00:00


Building a new DB, current time: 06/12/2022 02:02:56
New DB name:   C:\Users\lukshkumar\Desktop\Blast\DB
New DB title:  C:\Users\lukshkumar\Desktop\Blast\subject_SPX.txt
Sequence type: Protein
Deleted existing Protein BLAST database named C:\Users\lukshkumar\Desktop\Blast\DB
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.0066148 seconds.




Building a new DB, current time: 06/12/2022 02:03:09
New DB name:   C:\Users\lukshkumar\Desktop\Blast\DB
New DB title:  C:\Users\lukshkumar\Desktop\Blast\subject_AAPL.txt
Sequence type: Protein
Deleted existing Protein BLAST database named C:\Users\lukshkumar\D

## Minute Data Blast Prediction

In [4]:
# Number of letters in re-indexed columns
number_of_letters = 0
for each_MA_list in list_of_MA_list:
    number_of_letters += len(each_MA_list) 

blast_results_metric = "Identity"

blast_index_start = 100
blast_index_end = 200

folder_path = "Tickers-Minute-Data/"
blast_folder_path = "Blast/"

if(run_for_all_tickers_within_a_directory):
    data_files = os.listdir(folder_path)
else:
    for each_file_index_in_directory in range(len(data_files)):
        data_files[each_file_index_in_directory] = data_files[each_file_index_in_directory] + ".csv"

subject_files = os.listdir(blast_folder_path)
#Reading the Blast-Tickers File

blast_tickers = pd.read_csv("Blast-Tickers.csv")
blast_tickers = blast_tickers["Tickers"].values.tolist()
print("Blast Ticker Received From File: ", blast_tickers)

for data_file in data_files:

    print("---------------- Filename: " + data_file + " ----------------")
    symbol_name = data_file.replace(".csv","")
    df_daily_data = pd.read_csv(folder_path + data_file)

    if(take_entire_data_for_subject):
        subject_starting_date = str(df_daily_data.loc[0, "Date"])
        subject_ending_date = str(df_daily_data.loc[len(df_daily_data.index) - 1, "Date"])
        print("Subject Start Date: ", subject_starting_date)
        print("Subject Ending Date: ", subject_ending_date)

    #Making a separate folder for each ticker.
    if(not os.path.isdir(blast_folder_path + "Tickers/" + symbol_name)):
        os.mkdir(blast_folder_path + "Tickers/" + symbol_name)
    blast_resultant_filename = blast_folder_path + "Tickers/" + symbol_name + "/results_query_" + symbol_name
    current_subject_filename = "subject_" + symbol_name + ".txt"
    current_query_filename = "query_" + symbol_name + ".txt"

    df_daily_data = df_daily_data.astype({"Date": str})

    getcustom_global_re_indexing(df_daily_data,number_of_letters)

    MA_list_count = 1
    for each_MA_list in list_of_MA_list:

        list_of_each_MA_value_column_name = []
        each_MA_list_sequence_name = "sequence" + str(MA_list_count)
        for each_MA_value in each_MA_list:

            each_MA_value_column_name = str(each_MA_value) + "_day_sma"
            list_of_each_MA_value_column_name.append(each_MA_value_column_name)
            #SMA Daily
            df_daily_data[each_MA_value_column_name] = SimpleMovingAvg(df_daily_data, "Close", each_MA_value)

        df_daily_data[each_MA_list_sequence_name] = get_sequence(df_daily_data,list_of_each_MA_value_column_name,["A", "B", "C", "D", "E"])
        MA_list_count += 1

    get_custom_symbol_parsed_re_indexing(df_daily_data,number_of_letters)

    # Merging all list of MA columns into one single column named as sequence
    consolidated_sequence = []
    for i in range(len(df_daily_data)):
        each_row_sequence = ""
        for j in range(1, len(list_of_MA_list)+1):
            j_column_name = "sequence" + str(j)
            each_row_sequence += df_daily_data.loc[i, j_column_name]
        consolidated_sequence.append(each_row_sequence)

    df_daily_data["sequence"] = consolidated_sequence

    #df_daily_data.to_csv("Testing-" + symbol_name +".csv")

    #getquery_blast_txt_file (data,symbol,column,start_date,end_date)
    total_rows_in_query_dataframe = getquery_blast_txt_file (df_daily_data,symbol_name,"sequence",query_starting_date, query_ending_date)
    print("Total Rows in Query: ", total_rows_in_query_dataframe)
    number_of_predicted_entries = int(total_rows_in_query_dataframe / ratio_of_predicted_entries)
    print("Predicted Rows: ", number_of_predicted_entries)

    #getsubject_blast_txt_file (data,symbol,column,start_date,end_date)
    getsubject_blast_txt_file (df_daily_data,symbol_name,"sequence",subject_starting_date, subject_ending_date, query_starting_date, query_ending_date)

    #BLAST INTEGRATION - FOR CURRENT SYMBOL
    !makeblastdb -in C:\Users\lukshkumar\Desktop\Blast\$current_subject_filename -dbtype prot -out C:\Users\lukshkumar\Desktop\Blast\DB
    !blastp -query C:\Users\lukshkumar\Desktop\Blast\$current_query_filename -db C:\Users\lukshkumar\Desktop\Blast\DB -out C:\Users\lukshkumar\Desktop\Blast\output.txt -outfmt 6 -max_hsps $top_n_blast_results

    blast_result_csv_file = pd.DataFrame(columns = ["Ticker", "Query", "Subject", "Identity", "Alignment Length", "MisMatches", "Gap Opens", "Query Start", "Query End", "Subject Start", "Subject End", "Evalue", "Bit Score"])
    file = open(blast_folder_path  + "output.txt","r")
    lines = file.readlines()
    file.close()
    for line in lines:
        blast_file_result = line.replace("\n","").split("\t")
        blast_file_result.insert(0,symbol_name)
        blast_result_csv_file.loc[len(blast_result_csv_file.index)] = blast_file_result

    #BLAST INTEGRATION - FOR OTHER SYMBOLS IN THE LIST
    for blast_ticker in blast_tickers:
        if(blast_ticker != symbol_name):
            blast_subject_filename = "subject_" + blast_ticker + ".txt"

            if(os.path.isfile(blast_folder_path +  blast_subject_filename)):
                # MAKE BLAST DB FROM THE SUBJECT
                !makeblastdb -in C:\Users\lukshkumar\Desktop\Blast\$blast_subject_filename -dbtype prot -out C:\Users\lukshkumar\Desktop\Blast\DB
                # RUN BLAST TO FIND THE MATCH
                !blastp -query C:\Users\lukshkumar\Desktop\Blast\$current_query_filename -db C:\Users\lukshkumar\Desktop\Blast\DB -out C:\Users\lukshkumar\Desktop\Blast\output.txt -outfmt 6 -max_hsps $top_n_blast_results
                # APPEND DATA IN CSV FILE
                file = open(blast_folder_path  + "output.txt","r")
                lines = file.readlines()
                file.close()
                for line in lines:
                    blast_file_result = line.replace("\n","").split("\t")
                    blast_file_result.insert(0,blast_ticker)
                    blast_result_csv_file.loc[len(blast_result_csv_file.index)] = blast_file_result


    blast_result_csv_file[blast_results_metric] = pd.to_numeric(blast_result_csv_file[blast_results_metric])
    blast_result_csv_file = blast_result_csv_file.sort_values(by=[blast_results_metric], ascending = False).reset_index(drop=True)
    blast_result_csv_file.to_csv(blast_resultant_filename + ".csv", index = False)

    #Creating another column for the filename for easier reference.
    blast_result_csv_file["filename"] = ""
    #Making sure that the top_n_blast_results are greater than the number of results in blasts file in case blast file has 
    #One result only and top results are two then it should not break.
    if(top_n_blast_results > blast_result_csv_file.shape[0]):
        top_n_blast_results = blast_result_csv_file.shape[0]
    for i in range(top_n_blast_results):
        print("I: ", i)
        blast_ticker_name = blast_result_csv_file.iloc[i,0]
        blast_identity_percentage = round(blast_result_csv_file.iloc[i,3], 0)
        blast_index_start = int(blast_result_csv_file.iloc[i,9])
        blast_index_end = int(blast_result_csv_file.iloc[i,10])

        #print("Blast Index Start: ", blast_index_start)

        source_dataframe  = pd.read_csv(folder_path + blast_ticker_name + ".csv")
        # Appending the data from Matched Ticker to the Target Ticker Data File.
        target_dataframe = get_how_to_append_end_data(source_dataframe,"Close",math.ceil(blast_index_end/number_of_letters) + 1,math.ceil(blast_index_end/number_of_letters) + number_of_predicted_entries + 2,df_daily_data,df_daily_data.shape[0] - 1,True)

        # Adding additional fields in target dataframe (predicted data). 
        target_dataframe["Open"] = target_dataframe["Close"]
        target_dataframe["High"] = target_dataframe["Close"]
        target_dataframe["Low"] = target_dataframe["Close"]
        target_dataframe["Volume"] = 0
        target_dataframe["Date"] = target_dataframe["Date"].dt.strftime('%Y%m%d')

        #Dropping the first two unnecessary records from the target dataframe that were only used to create predicted data.
        target_dataframe = target_dataframe.iloc[2:,:]

        consolidated_filename_date_portion = str(target_dataframe.loc[0, "Date"])
        # Appending the source data into target dataframe in order to make up a combinded file having previous and predicted data.
        consolidated_source_plus_target_df = source_dataframe.append(target_dataframe)
        consolidated_source_plus_target_df.reset_index(drop = True)

        # Date Against the Index Matched from Blast
        df_date_we_match_found = get_date_from_re_indexing(source_dataframe,(math.ceil(blast_index_start/number_of_letters) -1), (math.ceil(blast_index_end/number_of_letters)))

        consolidated_source_plus_target_filename = symbol_name + consolidated_filename_date_portion + "M" + str(ratio_of_predicted_entries) + "V" + str(i + 1) + ".csv"
        consolidated_source_plus_target_filename_with_path = blast_folder_path + "Tickers/" + symbol_name + "/" + consolidated_source_plus_target_filename

        blast_result_csv_file.loc[i, "filename"] =  consolidated_source_plus_target_filename

        consolidated_source_plus_target_df.to_csv(consolidated_source_plus_target_filename_with_path, index = False)

        print("Match is Found in: ", blast_ticker_name, "    Ratio: ", blast_identity_percentage)
        print("Date Start: ", df_date_we_match_found.iloc[0])
        print("Date End: ", df_date_we_match_found.iloc[1])
        print("-----------------------------------------------------------------------------")

#Saving the updated file with Date added in the filename so that we have a separate blast result file for each day.
blast_result_csv_file.to_csv(blast_resultant_filename + "_" + consolidated_filename_date_portion + "_M" + ".csv", index = False)
#Removing the generic file created above to make sure we don't have duplicate data.
os.remove(blast_resultant_filename + ".csv")

print("---------------- Program Execution Completed Successfully ----------------")

Blast Ticker Received From File:  []
---------------- Filename: AAPL.csv ----------------
Subject Start Date:  20220331
Subject Ending Date:  20220517
Total Rows in Query:  4609
Predicted Rows:  460
Trimmed Subject Starting Date:  2022-03-31 00:00:00
Trimmed Subject Ending Date:  2022-04-30 00:00:00


Building a new DB, current time: 06/05/2022 04:31:49
New DB name:   C:\Users\lukshkumar\Desktop\Blast\DB
New DB title:  C:\Users\lukshkumar\Desktop\Blast\subject_AAPL.txt
Sequence type: Protein
Deleted existing Protein BLAST database named C:\Users\lukshkumar\Desktop\Blast\DB
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.0019837 seconds.


I:  0
Match is Found in:  AAPL     Ratio:  47.0
Date Start:  20220405
Date End:  20220419
-----------------------------------------------------------------------------
I:  1
Match is Found in:  AAPL     Ratio:  45.0
Date Start:  20220409
Date End:  20220428
----------------------------------------------